In [ ]:
import cv2
import numpy as np
import json
import os
from pathlib import Path

In [ ]:
dataset_path = "/media/notingcode/Data/Projects/gaussian-splatting/8cam/breaking_dance_real/images"
undistort_destination_path = "/media/notingcode/Data/Projects/gaussian-splatting/8cam/breaking_dance_real/undistorted"
params_json_path = "/media/notingcode/Data/Projects/gaussian-splatting/8cam/cam_params.json"

In [ ]:
dataset_subdirs = os.listdir(dataset_path)

In [ ]:
camera_params = json.load(open(params_json_path))

In [ ]:
for subdir in dataset_subdirs:
    subdir_idx = int(subdir.replace("cam",""))
    subdir_absolute_path = Path(os.path.join(dataset_path,subdir))
    images_dir_sorted = sorted([image_path for image_path in subdir_absolute_path.glob("*.jpg")])
    
    camera_name = None
    
    for key in camera_params.keys():
        if str(subdir_idx) in key:
            camera_name = key
    
    for image_dir in images_dir_sorted:
        img = cv2.imread(image_dir.as_posix())
        h,w = img.shape[:2]
        
        intrinsic = np.array(camera_params[camera_name]['in'])
        distortion = np.array(camera_params[camera_name]['dist'])
        
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(intrinsic, distortion, (w,h), 1, (w,h))
        
        dst = cv2.undistort(img, intrinsic, distortion, None, newcameramtx)
        
        x,y,w,h = roi
        
        dst = dst[y:y+h, x:x+w]
        
        cv2.imwrite(os.path.join(undistort_destination_path, image_dir.name), dst)